In [8]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness, context_utilization, noise_sensitivity_relevant
from ragas import evaluate
from datasets import Dataset
import os
import nest_asyncio
nest_asyncio.apply()


os.environ["OPENAI_API_KEY"] = ""

def determine_metrics(user_query, assistant_response, context, ground_truth):
    metrics = []

    # Check conditions and append appropriate metrics
    if user_query is not None and assistant_response is not None and context is not None:
        metrics.append(faithfulness)
        metrics.append(answer_relevancy)
        metrics.append(context_utilization) #precision @k

    if user_query is not None and assistant_response is not None and context is not None and ground_truth is not None:
        # metrics.append(context_precision)
        metrics.append(context_recall)
        # metrics.append(noise_sensitivity_relevant)

    if user_query is not None and assistant_response is not None and ground_truth is not None:
        metrics.append(answer_similarity)
        metrics.append(answer_correctness)

    if context is not None and ground_truth is not None:
        metrics.append(context_entity_recall)

    return metrics


def std_evaluate(user_query=None, assistant_response=None, context=None, ground_truth=None):
    # Initialize the dictionary
    data = {}

    # Add key-value pairs to the dictionary if the corresponding variables are not None
    if user_query is not None:
        data['question'] = [user_query]

    if assistant_response is not None:
        data['answer'] = [assistant_response]

    if context is not None:
        data['contexts'] = [context]

    if ground_truth is not None:
        data['ground_truth'] = [ground_truth]
    metrics = determine_metrics(user_query, assistant_response, context, ground_truth)
    
    dataset = Dataset.from_dict(data)
    score = evaluate(dataset,metrics=metrics)
    return score



In [17]:
user_query = "What is the weather today?"
assistant_response = "The weather today is CLOUDY (5°C)"
context = ["The user asked about today's weather.", "weather is CLOUDY", "temprature is 25°C."]
ground_truth = "The weather forecast for today is sunny with a high of 25°C."
std_evaluate(user_query = user_query, assistant_response= assistant_response, ground_truth=None, context=context)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'faithfulness': 0.5000, 'answer_relevancy': 0.9767, 'context_utilization': 0.5000}

In [ ]:
# faithfulness, answer_relevancy,  context_utilization      -> user_query, assistent_response, context
# context_precision     -> user_query, assistent_response, context, ground_truth
# answer_similarity, answer_correctness -> user_query, assistent_response, ground_truth
# context_entity_recall                 -> context, ground_truth